<a href="https://colab.research.google.com/github/Snoke9/Online-shop/blob/main/clustering_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 5
# Решение задачи кластеризации
**Цель работы:** научиться производить кластерный анализ данных с использованием метода К-средних.

## Специальности колледжа (мужчины/женщины)

научиться производить кластерный анализ данных с использованием метода К-средних.

Атрибуты:
* **Rank** – рейтинг по медианному доходу
* **Major_code** – основной код, FO1DP в ACS PUMS
* **Major** – специальность
* **Major_category** – категория специальности
* **total** – общее количество людей
* **Men** – выпускники мужского пола
* **Women** – выпускники женского пола
* **ShareWoman** – доля женщин в общем числе
* **Median** – средний заработок штатных работников, работающих круглый год





In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/women-stem.csv')
df.head()

## Получим информацию о датасете

In [ ]:
df.info()

## Проверяем данные на наличие пропущенных значений и выбросов

In [ ]:
# проверка на пропущенные значения
print(df.isnull().sum())

## Удаляем невлияющие категориальные признаки

In [ ]:
df_copy = df.drop(columns=['Rank', 'Major_code', 'Major', 'Major_category'])
df_copy.head()

## Применяем операцию нормализации для численной устойчивости

In [ ]:
# импортируем класс для стадартизации данных
from sklearn.preprocessing import StandardScaler

# создадим объект класса StandarlScaler

scaler = StandardScaler()
scaler

## Приведем данные к единому масштабу

In [ ]:
X = scaler.fit_transform(df_copy)
X[:5]

## Определяем оптимальное количество кластеров с помощью метода локтя

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
wcss = []
for i in range(1, 11):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
  kmeans.fit(X)
  wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# заново обучим модель с пятью кластерами
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=42)

# обучаем модель на стандартизированных данных
kmeans.fit(X)

## Предсказываем кластеры и визуализируем результаты

In [ ]:
# предсказываем кластеры для каждого
labels = kmeans.predict(X)

In [ ]:
# визуализируем результаты
plt.scatter(X[:, 0], X[:, 3], c=labels, cmap='viridis')
plt.title('Специальности колледжа')
plt.xlabel('Признак 0')
plt.ylabel('Признак 3')
plt.show()

## Интерпретируем результаты

In [ ]:
labels

In [ ]:
df_copy['Label'] = labels
df_copy.head()

In [ ]:
df_copy.groupby('Label').mean()

В кластере 0 наименьшая доля женщин, большое количество выпускников и близкая к самой высокой средняя зарплата. В кластере 1 преобладают женщины, низкое количество выпускником и наименьшая средняя зарплата. В кластере 2 наибольшее количество выпускников, наименьшая доля мужчин и близкая к низкой средняя зарплата. В кластере 3 наименьшее общее количество выпускников, наибольшая доля мужчин и наибольшая средняя зарплата.

In [ ]:
plt.scatter(X[labels == 0, 0], X[labels == 0, 3], s=100, c='red', label='Cluster 0')
plt.scatter(X[labels == 1, 0], X[labels == 1, 3], s=100, c='green', label='Cluster 1')
plt.scatter(X[labels == 2, 0], X[labels == 2, 3], s=100, c='blue', label='Cluster 2')
plt.scatter(X[labels == 3, 0], X[labels == 3, 3], s=100, c='purple', label='Cluster 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 3], s=300,
            c='yellow', label='Centroids')
plt.title('Кластеры специальностей колледжа')
plt.xlabel('Общее количество людей')
plt.ylabel('Доля женщин в общем числе')
plt.legend()
plt.show()

In [ ]:
plt.scatter(X[labels == 0, 2], X[labels == 0, 4], s=100, c='red', label='Cluster 0')
plt.scatter(X[labels == 1, 2], X[labels == 1, 4], s=100, c='green', label='Cluster 1')
plt.scatter(X[labels == 2, 2], X[labels == 2, 4], s=100, c='blue', label='Cluster 2')
plt.scatter(X[labels == 3, 2], X[labels == 3, 4], s=100, c='purple', label='Cluster 3')
plt.scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 4], s=300,
            c='yellow', label='Centroids')
plt.title('Кластеры специальностей колледжа')
plt.xlabel('Выпускники женского пола')
plt.ylabel('Средний заработок штатных работников')
plt.legend()
plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Уменьшаем размерность до 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Визуализация
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Cluster of specialities')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
# Добавляем названия специальностей в исходные данные
df_copy['Major'] = df['Major']
df_copy.head()

In [ ]:
# Вывод специальностей в каждом кластере
for cluster_num in range(4):
  print(f'\nCluster {cluster_num} specialties:')
  print(df_copy[df_copy['Label'] == cluster_num]['Major'].values)